# Inpainting on memes
---

This notebook aims to do inpainting on existing memes in order to remove the text and obtain a meme template. Inpainting is done on texts with background using a pre-trained model provided by: https://github.com/JiahuiYu/generative_inpainting and called in inpaint_model.py. Character segmentation is done to generate masks required for inpainting. Texts on white or coloured background will be replaced with a simple filled rectangle. Output is saved into './images/test/{imagename}_output.jpg'.

---
### Data

Meme images should be saved under './images/test/{imagefile}' and a csv file containing the following columns should be saved under './images/test_labels.csv'. xmin, ymin, xmax, ymax make up the coordinates of bounding boxes around the text:
* filename 
* xmin
* ymin
* xmax
* ymax

---
### Essential Libraries

As Tensorflow version 1.x is required, ensure that your Python version is 3.7.x and below. 
> Pandas : Library for Data Acquisition and Preparation  
> NumPy : Library for Numeric Computations in Python <br>
> OpenCV : Library for computer vision algorithms <br>
> Tensorflow : Library for building advanced machine learning models <br>
> Pillow : Python Imaging Library for image processing capabilities <br>
> Neuralgym : Library from https://github.com/JiahuiYu/neuralgym

## 1. Install and import relevant libraries

In [1]:
from platform import python_version

print(python_version())

3.7.10


In [2]:
pip install 'tensorflow==1.15.2'

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install git+https://github.com/JiahuiYu/neuralgym

  Cloning https://github.com/JiahuiYu/neuralgym to /private/var/folders/5m/wcsj7dr53tx27vnf1jlqpt840000gn/T/pip-req-build-29aap39e
  Running command git clone -q https://github.com/JiahuiYu/neuralgym /private/var/folders/5m/wcsj7dr53tx27vnf1jlqpt840000gn/T/pip-req-build-29aap39e
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFilter
import neuralgym as ng
import os
import operator

from inpaint_model import InpaintCAModel

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






## 2. Obtain bounding boxes from CSV

In [70]:
chineseDF = pd.read_csv('./images/templates/good_chinese_inpaint.csv')
chineseDF

,meme_id,xmin,ymin,xmax,ymax
0,9814,15,57,94,68
1,9814,414,93,479,117
2,9814,129,316,210,343
3,9814,134,117,296,134
4,9814,352,286,455,306
...,...,...,...,...,...
70,5663,42,526,714,636
71,5163,43,28,900,77
72,5163,403,171,902,230
73,5163,485,643,818,686


In [2]:
labelsDF = pd.read_csv('./images/templates/textbox.csv')

In [3]:
labelsDF

,meme_id,box_id,xmin,ymin,xmax,ymax,text,updated_text
0,0,0,140,222,373,235,"gaporeans,we must judge a person no",NaN
1,0,1,176,239,350,250,his educational qualifications,NaN
2,0,2,26,303,498,335,ALLTHECIVILSERVANTSARELIKE,NaN
3,0,3,28,361,110,405,SGAG,NaN
4,0,4,109,373,408,462,ISMELL,NaN
...,...,...,...,...,...,...,...,...
133925,14897,26,629,869,809,920,Memedef,NaN
133926,14898,0,236,35,601,76,that one cbkia,NaN
133927,14898,1,506,154,764,177,wait need refill bottle,NaN
133928,14898,2,17,160,378,191,oh no everyone fall-in alr,NaN


In [18]:
def get_mask(image, mask, meme_id):
    inpainting_required = False
    for index, row in labelsDF.iterrows():
        
              
        if(row['meme_id'] == meme_id):
            
            left = row['xmin']
            top = row['ymin']
            right = row['xmax']
            bottom = row['ymax']

            padding = get_padding(image, left, top, right, bottom)
            # check if area around bounding box is of similar colour
            scene_bg = check_bg(padding, image, imagefile, left, top, right, bottom)
            
            if scene_bg: 
                # draw text characters on the mask for inpainting
                mask = get_contours(mask, left, top, right, bottom)
                inpainting_required = True
            else:  # single colour background
                topleft = (left,top)
                bottomright = (right,bottom)
                # directly draw a filled rectangle on image
                colour = tuple(int(x) for x in padding[4,4])
                image = cv2.rectangle(image, topleft, bottomright, colour, -1)
            
    return image, mask, inpainting_required

## 3. Check for single colour backgrounds

In [5]:
def crop_img(image, left, top, right, bottom):
    h, w = image.shape[:2]
    # ensure values are not out of image
    left2 = max(0, left-5)
    top2 = max(0, top-5)
    right2 = min(w, right+5)
    bottom2 = min(h, bottom+5)

    # crop bounding box with some padding
    cropped_img = image[top2:bottom2, left2:right2].copy()
    
    return cropped_img, left2, top2, right2, bottom2

In [6]:
def get_padding(image, left, top, right, bottom):
    bbox, left2, top2, right2, bottom2 = crop_img(image, left, top, right, bottom)
    
    # turn the pixel values of the bounding box to white 
    padding = bbox.copy()
    padding[top-top2:bottom-bottom2, left-left2:right-right2] = (255, 255, 255)

    return padding

In [7]:
def check_bg(padding, image, imagefile, left, top, right, bottom):
    
    # colour of one of the bg pixel in bgr
    colour = padding[4,4]
    
    max_colour = np.add(colour, (25,25,25))
    max_colour = [min(255, x) for x in max_colour]
    min_colour = np.subtract(colour, (25,25,25))
    min_colour = [max(0, x) for x in min_colour]
    
    upperbound = np.full(padding.shape, max_colour, dtype="uint8")
    lowerbound = np.full(padding.shape, min_colour, dtype="uint8")
    
    thresh = np.zeros(padding.shape, dtype="uint8")
    thresh = cv2.inRange(padding, lowerbound, upperbound)
    # thresh set to 255 if padding within range and 0 otherwise
    
    thresh[5 : -5, 5 : -5] = 255
    
    pixels = cv2.countNonZero(thresh)  # number of pixels within range
    H, W = padding.shape[:2]
    
    # print(pixels)
    # print(H*W)
    
    if pixels >= 0.98 * (H * W):
        print('Coloured Background')
        return False
    
    else:
        print('Scene Background')
        return True

## 4. Character segmentation

In [28]:
def get_contours(mask, left, top, right, bottom):
    bbox, left2, top2, right2, bottom2 = crop_img(image, left, top, right, bottom)
    
    cv2.imshow('bbox', bbox)
    cv2.waitKey(0)
    
    clear_img = cv2.fastNlMeansDenoisingColored(bbox,None,10,10,7,21)
    gray_img = cv2.cvtColor(clear_img,cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('Gray Image', gray_img)
    cv2.waitKey(0)

    thresh_img = cv2.threshold(gray_img, 150, 255, cv2.THRESH_BINARY)[1] 
    
    contours, hierarchy = cv2.findContours(thresh_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 

    drawing = np.zeros(bbox.shape, dtype="uint8")
    cv2.drawContours(drawing, contours, -1, (255,255,255), -1) 
    # third argument = -1 to draw all contours, last argument = -1 to fill contour
    
    # draw outline around contour depending on height of bbox
    bbox_h , bbox_w = bbox.shape[:2]
    thickness = max((bbox_h//3), 2)
    # print(f"bbox height is {bbox_h}, thickness is {thickness}")
    cv2.drawContours(drawing, contours, -1, (255,255,255), thickness)
    
    mask[top2:bottom2,left2:right2,:] = drawing
    
    return mask

## 5. Obtain model

In [9]:
FLAGS = ng.Config('inpaint.yml')
model = InpaintCAModel()

---------------------------------- APP CONFIG ----------------------------------
num_gpus_per_job: 1
num_cpus_per_job: 4
num_hosts_per_job: 1
memory_per_job: 32
gpu_type: nvidia-tesla-p100
name: places2_gated_conv_v100
model_restore: 
dataset: memes
random_crop: False
val: False
log_dir: logs/full_model_celeba_hq_256
gan: sngan
gan_loss_alpha: 1
gan_with_mask: True
discounted_mask: True
random_seed: False
padding: SAME
train_spe: 4000
max_iters: 100000000
viz_max_out: 10
val_psteps: 2000
data_flist: 
  memes: ['data/train_shuffled.flist', 'data/validation_shuffled.flist']
static_view_size: 30
img_shapes: [256, 256, 3]
height: 128
width: 128
max_delta_height: 32
max_delta_width: 32
batch_size: 16
vertical_margin: 0
horizontal_margin: 0
ae_loss: True
l1_loss: True
l1_loss_alpha: 1.0
guided: False
edge_threshold: 0.6
--------------------------------------------------------------------------------


In [10]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True

## 6. Image and mask pre-processing

In [11]:
def preprocessing(image, mask):
    h, w = image.shape[:2]
    grid = 8
    image = image[:h//grid*grid, :w//grid*grid, :]
    mask = mask[:h//grid*grid, :w//grid*grid, :]

    image = np.expand_dims(image, 0)
    mask = np.expand_dims(mask, 0)
    input_image = np.concatenate([image, mask], axis=2)
    
    return input_image

## 7. Inpainting

In [12]:
def inpainting(input_image):
    with tf.Session(config=sess_config) as sess:
        input_constant = tf.constant(input_image, dtype=tf.float32)
        # output = model.build_server_graph(FLAGS, input_image)
        output = model.build_server_graph(FLAGS, input_constant, reuse=tf.AUTO_REUSE)
        output = (output + 1.) * 127.5
        output = tf.reverse(output, [-1])
        output = tf.saturate_cast(output, tf.uint8)
        # load pretrained model
        vars_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        assign_ops = []
        for var in vars_list:
            vname = var.name
            from_name = vname
            var_value = tf.contrib.framework.load_variable('./model_logs/release_places2_256', from_name)
            assign_ops.append(tf.assign(var, var_value))
        sess.run(assign_ops)
        print('Model loaded.')
        result = sess.run(output)
        
        # create an Image Object to apply smooth filter
        print('Smooth image.')
        image_obj = Image.fromarray(result[0][:, :, ::], 'RGB')
        smoothenedImage = image_obj.filter(ImageFilter.SMOOTH)

        output_img = np.array(smoothenedImage)
        output_img = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)
        return result[0][:, :, ::-1]

## 8a. Run batch images

In [13]:
memesDF = pd.read_csv('./images/templates/meme.csv')
memesDF

,meme_id,filename,platform,perceptual_hash,vector_id
0,0,1000046440010344_0.jpg,sgag,dummy,-1
1,1,1000253469989641_0.jpg,sgag,dummy,-1
2,2,1000373609977627_0.jpg,sgag,dummy,-1
3,3,1000388496642805_0.jpg,sgag,dummy,-1
4,4,1000432826638372_0.jpg,sgag,dummy,-1
...,...,...,...,...,...
14894,14894,3869683853069014_0.jpg,memedef,dummy,-1
14895,14895,3877032432334156_0.jpg,memedef,dummy,-1
14896,14896,3879765992060800_0.jpg,memedef,dummy,-1
14897,14897,3886148051422594_0.jpg,memedef,dummy,-1


In [139]:
templatesDF = pd.read_csv('./images/templates/templates.csv', index_col=False)
templatesDF
# templatesDF = templatesDF.sort_values('original_meme_id')
# templatesDF.to_csv('./images/templates/templates/templates.csv', index=False)

,original_meme_id,template_id
0,5163,5163_template
1,5663,5663_template
2,6016,6016_template
3,6179,6179_template
4,7217,7217_template
...,...,...
93,100,100_template
94,101,101_template
95,102,102_template
96,108,108_template


In [ ]:
# templatesDF = pd.read_csv('./images/templates/templates/templates.csv', index_col=False)

for x in range(33,34):
    imagefile = memesDF.iloc[x]['filename']
    #imagefile = "2686512184719526_0.jpg"
    imagename = imagefile.split('.')[0]
    image = cv2.imread(f'../new-meme-observatory/public/meme_images/dataset/{imagefile}')
    
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    
    mask = np.zeros(image.shape, dtype="uint8")

    output_img, mask, inpainting_required = get_mask(image.copy(), mask, x)
    
    cv2.imshow('Mask', mask)
    cv2.waitKey(0)

#     if inpainting_required:
#         input_image = preprocessing(output_img, mask)
#         output_img = inpainting(input_image)
#         # input and output images the same size for stacking
#         # required_size = (image.shape[1], image.shape[0])
#         # output_img = cv2.resize(output_img, required_size)

#     # horizontal_stack = np.hstack((image, output_img))

#     # change output file accordingly
#     output_file = f'./images/templates/templates/{x}_template.jpg'
#     cv2.imwrite(output_file, output_img)

#     templatesDF.loc[len(templatesDF)] = [str(x), str(x) + "_template"]

# templatesDF.to_csv('./images/templates/templates/templates.csv', index=False)
        

Scene Background
Scene Background


In [43]:
directory = f'./images/test/'

for imagefile in os.listdir(directory):
    if imagefile.endswith(".jpg") and 'output' not in imagefile:
        imagename = imagefile.split('.')[0]
        print(imagename)
        image = cv2.imread(f'./images/test/{imagefile}')
        mask = np.zeros(image.shape, dtype="uint8")
        
        output_img, mask, inpainting_required = get_mask(image.copy(), mask, imagefile)
        
        if inpainting_required:
            input_image = preprocessing(output_img, mask)
            output_img = inpainting(input_image)
            # input and output images the same size for stacking
            required_size = (image.shape[1], image.shape[0])
            output_img = cv2.resize(output_img, required_size)
            
        horizontal_stack = np.hstack((image, output_img))
        
        # change output file accordingly
        output_file = f'./images/test/range25thick3/{imagename}_output.jpg'
        cv2.imwrite(output_file, horizontal_stack)
        

204745471
149017411
299381083


KeyboardInterrupt: 

## 8b. Run single image

In [31]:
imagename = '4540859889262297_0'
imagefile = f'{imagename}.jpg'
image = cv2.imread(f'../new-meme-observatory/public/meme_images/dataset/{imagefile}')

mask = np.zeros(image.shape, dtype="uint8")
output_img, mask, inpainting_required = get_mask(image.copy(), mask, 8908)

if inpainting_required:
    input_image = preprocessing(output_img, mask)
    output_img = inpainting(input_image)
    # input and output images the same size for stacking
#     required_size = (image.shape[1], image.shape[0])
#     output_img = cv2.resize(output_img, required_size)

# horizontal_stack = np.hstack((image, output_img))

# cv2.imshow('Horizontal Stack', horizontal_stack)
# cv2.waitKey()

# change output file accordingly
output_file = f'./images/templates/8908_baseline.jpg'
cv2.imwrite(output_file, output_img)

Scene Background
Coloured Background
Coloured Background
Scene Background
Coloured Background
Scene Background
Scene Background
Coloured Background
Coloured Background
Coloured Background
Model loaded.
Smooth image.


True

## 8c. Run with bounding boxes as mask

In [16]:
imagename = '4540859889262297_0'
imagefile = f'{imagename}.jpg'
image = cv2.imread(f'./images/templates/{imagefile}')

mask = np.zeros(image.shape, dtype="uint8")

for index, row in labelsDF.iterrows():
    if(row['meme_id'] == 8908):
        left = row['xmin']
        top = row['ymin']
        right = row['xmax']
        bottom = row['ymax']
        topleft = (left,top)
        bottomright = (right,bottom)
        cv2.rectangle(mask, topleft, bottomright, (255,255,255), -1)

cv2.imshow('Mask', mask)
cv2.waitKey()

input_image = preprocessing(image, mask)
output_img = inpainting(input_image)

# change output file accordingly
output_file = f'./images/test/baseline_inpainting/8908_output.jpg'
cv2.imwrite(output_file, output_img)

Model loaded.
Smooth image.


True

## 9. Miscellaneous Functions

#### Check if bounding box is correct

In [19]:
imagefile = '4540859889262297_0.jpg'
image = cv2.imread(f'./images/templates/{imagefile}')

for index, row in labelsDF.iterrows():
    if(row['meme_id'] == '8908'):
        left = row['xmin']
        top = row['ymin']
        right = row['xmax']
        bottom = row['ymax']

        # crop bounding box
        bbox, left2, top2, right2, bottom2 = crop_img(image, left, top, right, bottom)
        
        cv2.imshow("BBOX", bbox)
        cv2.waitKey(200)

#### Change values of csv

labelsDF.ymin += 3
labelsDF.ymax += 3
labelsDF.to_csv('./images/test_labels.csv', index=False, sep=',')